# Memory

<div style="display: flex; justify-content: flex-start; gap: 10px;">
  <img src="./assets/LC_Memory_after.png" style="width:300px; border:1px solid #ccc; border-radius:6px;">
</div>

Persisting messages, or 'agent state' between invocations of the agent.

## Setup

Load and/or check for needed environmental variables

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

OPENAI_API_KEY=<not set>
LANGSMITH_API_KEY=****2f66
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ials


In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [3]:
from dataclasses import dataclass

@dataclass
class RuntimeContext:
    db: SQLDatabase

In [4]:
from langchain_core.tools import tool
from langgraph.runtime import get_runtime

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [5]:
SYSTEM_PROMPT = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""

In [7]:
import os
from langchain_ollama import ChatOllama

model = ChatOllama(model=os.getenv("OLLAMA_MODEL") , temperature= 0 , base_url=os.getenv("OLLAMA_BASE_URL"))

In [8]:
from langsmith import uuid7
run_id = uuid7()  

In [9]:
from langchain.agents import create_agent

agent = create_agent(
    model=model,
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
)

## Repeated Queries

In [11]:
question = "This is Frank Harris, What was the total on my last invoice?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
    context=RuntimeContext(db=db),
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

This is Frank Harris, What was the total on my last invoice?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (c0ef5edb-da5c-493b-a265-4c501a1fb367)
 Call ID: c0ef5edb-da5c-493b-a265-4c501a1fb367
  Args:
    query: SELECT name FROM sqlite_master WHERE type='table';
================================= Tool Message =================================
Name: execute_sql

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]
================================== Ai Message ==================================
Tool Calls:
  execute_sql (18b7f0ab-bfef-4015-b004-27af7bb48b7a)
 Call ID: 18b7f0ab-bfef-4015-b004-27af7bb48b7a
  Args:
    query: SELECT InvoiceId, Total FROM Invoice WHERE CustomerId = (SELECT CustomerId FROM Customer WHERE FirstName='Frank' AND La

In [12]:
question = "What were the titles?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What were the titles?
================================== Ai Message ==================================

Could you let me know which titles you’re referring to? For example, are you asking about book titles, movie titles, song titles, or something else in the database? Once I know the context, I can pull the relevant list for you.


## Add memory

In [13]:
from langgraph.checkpoint.memory import InMemorySaver

In [14]:
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage

agent = create_agent(
    model=model,
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
    checkpointer=InMemorySaver(),
)

In [15]:
question = "This is Frank Harris, What was the total on my last invoice?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

This is Frank Harris, What was the total on my last invoice?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (048e69e3-914e-42c1-aff9-50d80d082eb0)
 Call ID: 048e69e3-914e-42c1-aff9-50d80d082eb0
  Args:
    query: SELECT name FROM sqlite_master WHERE type='table';
================================= Tool Message =================================
Name: execute_sql

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]
================================== Ai Message ==================================
Tool Calls:
  execute_sql (5d70cbf4-78cc-4ee6-9fb3-ec472711cdd9)
 Call ID: 5d70cbf4-78cc-4ee6-9fb3-ec472711cdd9
  Args:
    query: SELECT InvoiceId, Total FROM Invoice WHERE CustomerId = (SELECT CustomerId FROM Customer WHERE FirstName='Frank' AND La

In [16]:
question = "What were the titles?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

What were the titles?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (8025af1d-e676-4384-9602-1d24d3cbb81c)
 Call ID: 8025af1d-e676-4384-9602-1d24d3cbb81c
  Args:
    query: SELECT t.Title FROM InvoiceLine il JOIN Track t ON il.TrackId = t.TrackId WHERE il.InvoiceId = 374;
================================= Tool Message =================================
Name: execute_sql

Error: (sqlite3.OperationalError) no such column: t.Title
[SQL: SELECT t.Title FROM InvoiceLine il JOIN Track t ON il.TrackId = t.TrackId WHERE il.InvoiceId = 374;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
================================== Ai Message ==================================
Tool Calls:
  execute_sql (c236d326-6e2c-4ffe-9a6c-6a1dd057ae7f)
 Call ID: c236d326-6e2c-4ffe-9a6c-6a1dd057ae7f
  Args:
    query: PRAGMA table_info(Track);
====================

## Try your own queries
Now that there is memory, check the agents recall!

In [17]:
question = "Your Question Here?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

Your Question Here?
================================== Ai Message ==================================

Sure! What would you like to explore next? For example, would you like to see the album names for the tracks on your last invoice, or perhaps the total number of invoices Frank Harris has received?
